## Deep Learning
Within this notebook, simple models are created, hyperparameter tuning is performed and a comparison is done to an alternative model. 

In [1]:
#importing necessary packages
import keras
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Input
import pandas as pd
import numpy as np
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from keras.backend import clear_session
from keras.layers import Dense, Activation
from keras.models import Sequential
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.backend import clear_session
from keras.layers import Dense, Activation
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm

In [2]:
#setting the seed for reproducibility
cross_validation = KFold(n_splits = 3, shuffle = True, random_state = 10)

seed = 14
tf.random.set_seed(14)

In [3]:
#checking if GPU is being used 
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12165323573285605563
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15513944064
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4858839230617404781
physical_device_desc: "device: 0, name: Quadro P5000, pci bus id: 0000:02:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [4]:
#read dataframe from the Data Preparation Notebook 
import pandas as pd
df_complete3 = pd.read_pickle('df_complete3_v3.pickle')

In [5]:
#change types of the whole dataframe so that we do not run out of memory during the training process 
df_complete3 = df_complete3.astype(np.uint8)

In [5]:
df_complete3.shape

(100040, 45)

In [7]:
#split off target column
target_column = df_complete3['number of conflicts']
df_complete3.drop(columns = ['number of conflicts'], inplace = True)

In [8]:
df_complete3.columns

Index([                                       'gender development index (GDI)',
                                               'human development index (HDI)',
                                                                'health index',
                                                                'income index',
                                                           'educational index',
                                                 'expected years of schooling',
                                                      'mean years of scholing',
                                                             'life expectancy',
                                                    'GNI per capita (000'000)',
                                                        'population (000'000)',
                                                                        'Year',
        'Access to clean fuels and technologies for cooking (% of population)',
                                     'Ac

In [9]:
#perform train-test-split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_complete3, target_column, test_size = 0.08, random_state = 0)


In [10]:
#perform train-test-split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_complete3, target_column, test_size = 0.08, random_state = 0)

#normalize the data with a MinMaxScaler
#sc = MinMaxScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

In [11]:
#normalize the data with a StandardScaler - performance regarding simple model is better, so we continue with the Standard Scaler!
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

C:\Users\Shadow\anaconda3\envs\tf-keras-gpu-2\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\Shadow\anaconda3\envs\tf-keras-gpu-2\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\Shadow\anaconda3\envs\tf-keras-gpu-2\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
#imputing of missing values in case of missing values

#imputer = SimpleImputer(strategy='constant', fill_value=0)
#X_train = imputer.fit_transform(X_train)
#X_test = imputer.transform(X_test)

### Creating some simple models

First, we create some simple models to get an understanding of what type of model performs the best. We compare a simple model with only one layer, to one wider and one larger model. Then we also train a larger and wider model. The best performing one is passed on to the hyperparameter tuning.

In [ ]:
#create a simple model
def simple_model():
    clear_session()
    model = Sequential()
    del model
    model = Sequential()
    model.add(Dense(40, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    # Compile model
    model.compile(loss='mean_squared_error')
    return model

In [ ]:
#evaluate the simple model 
estimators = []
estimators.append(('keras_reg', KerasRegressor(model=simple_model, epochs=100, batch_size=2000
                                               , verbose=2)))

# fix random seed for reproducibility
seed = 14
tf.random.set_seed(seed)

pipeline = Pipeline(estimators)
results = cross_val_score(pipeline, X_train, y_train , cv= cross_validation, scoring='neg_mean_squared_error')
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -223.00 (2.27) MSE

In [ ]:
#create a larger model, so more layers
def larger_model():
    clear_session()
    model = Sequential()
    del model
    model = Sequential()
    model.add(Dense(40, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(20, kernel_initializer='normal', activation='relu'))
    model.add(Dense(10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    print(model.summary())
    # Compile model
    model.compile(loss='mean_squared_error')
    return model

In [ ]:
#evaluate the larger model
estimators = []
estimators.append(('keras_reg', KerasRegressor(model=larger_model, epochs=100, batch_size=2000)))


pipeline = Pipeline(estimators)
results = cross_val_score(pipeline, X_train, y_train , cv=cross_validation, scoring='neg_mean_squared_error')
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -153.28 (8.77) MSE -- this model performs better than the initial one. 

In [ ]:
#create a simple wider model
def wider_model():
    clear_session()
    model = Sequential()
    del model
    model = Sequential()
    
    model.add(Dense(40, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
    model.add(Dense(60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    # Compile model
    model.compile(loss='mean_squared_error')
    return model

In [ ]:
#evaluate the simple wider model
estimators = []
estimators.append(('keras_reg', KerasRegressor(model=wider_model, epochs=100, batch_size=2000
                                               , verbose=2)))
pipeline = Pipeline(estimators)
results = cross_val_score(pipeline, X_train, y_train , cv=cross_validation, scoring='neg_mean_squared_error')
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -158.88 (11.24) MSE -- this model performs slightly worse than the larger one. 

In [ ]:
#create a combination of the wider and larger model
def wider_larger_model():
    clear_session()
    model = Sequential()
    del model
    model = Sequential()
    
    model.add(Dense(40, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
    model.add(Dense(60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(50, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(20, kernel_initializer='normal', activation='relu'))
    model.add(Dense(10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    # Compile model
    model.compile(loss='mean_squared_error')
    return model

In [ ]:
#evaluate the wider and larger model
estimators = []
estimators.append(('keras_reg', KerasRegressor(model=wider_larger_model, epochs=100, batch_size=2000
                                               , verbose=2)))
pipeline = Pipeline(estimators)
results = cross_val_score(pipeline, X_train, y_train , cv= cross_validation, scoring='neg_mean_squared_error')
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -143.62 (4.16) MSE  -- this model performs the best out of all simple models.

**Finding**: wider_larger_model performs best, so we use this model for the following hyperparameter tuning. 

For the best model, we try the MinMaxScaler and the StandardScaler. 

In [ ]:
#perform train-test-split
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(df_complete3, target_column, test_size = 0.08, random_state = 0)

#normalize the data with a MinMaxScaler
#sc = MinMaxScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

In [ ]:
#evaluate the wider and larger model with MinMaxScaler (instead of StandardScaler)
estimators = []
estimators.append(('keras_reg', KerasRegressor(model=wider_larger_model, epochs=100, batch_size=2000
                                               , verbose=2)))
pipeline = Pipeline(estimators)
results = cross_val_score(pipeline, X_train, y_train , cv= cross_validation, scoring='neg_mean_squared_error')
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -236.03 (103.82) MSE -- performance of StandardScaler is better than the MinMaxScaler. 

### Hyperparameter Tuning

Hyperparameter Tuning is performed to further improve the performance of the neural network. 

Note: as part of the hyperparameter tuning, the epochs get reduced to 50 to speed up the training process.

1) Optimizer - SGD, RMSprop and Adam

In [11]:
#define the wider_larger_model
def wider_larger_model():
    clear_session()
    model = Sequential()
    del model
    model = Sequential()
    
    model.add(Dense(40, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
    model.add(Dense(60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(50, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(20, kernel_initializer='normal', activation='relu'))
    model.add(Dense(10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    # Compile model
    model.compile(loss='mean_squared_error')
    return model

In [19]:
#grid search for optimizer
# create model
model = KerasRegressor(model=wider_larger_model, verbose=1, epochs = 50, batch_size = 2000, loss = "mean_squared_error")

# fix random seed for reproducibility
seed = 14
tf.random.set_seed(seed)

# define the grid search parameters
optimizer = ["SGD", "RMSprop", "Adam"]

param_grid = dict(optimizer = optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv= cross_validation, scoring = "neg_mean_squared_error")
grid_result = grid.fit(X_train, y_train, validation_data = (X_test, y_test))

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
31/31 [==============================] - 2s 21ms/step - loss: 374.6258 - val_loss: 287.0601
Epoch 2/50
31/31 [==============================] - 0s 13ms/step - loss: 327.2076 - val_loss: 266.5721
Epoch 3/50
31/31 [==============================] - 0s 13ms/step - loss: 313.4785 - val_loss: 258.5006
Epoch 4/50
31/31 [==============================] - 0s 13ms/step - loss: 306.1486 - val_loss: 253.5956
Epoch 5/50
31/31 [==============================] - 0s 12ms/step - loss: 301.4651 - val_loss: 250.9155
Epoch 6/50
31/31 [==============================] - 0s 12ms/step - loss: 297.3497 - val_loss: 246.2269
Epoch 7/50
31/31 [==============================] - 0s 13ms/step - loss: 292.0669 - val_loss: 242.9546
Epoch 8/50
31/31 [==============================] - 0s 13ms/step - loss: 285.5938 - val_loss: 235.7146
Epoch 9/50
31/31 [==============================] - 0s 14ms/step - loss: 279.0816 - val_loss: 228.2328
Epoch 10/50
31/31 [==============================] - 0s 12ms/step - loss:

31/31 [==============================] - 0s 14ms/step - loss: 289.4162 - val_loss: 240.7607
Epoch 8/50
31/31 [==============================] - 0s 14ms/step - loss: 283.5168 - val_loss: 234.2177
Epoch 9/50
31/31 [==============================] - 0s 14ms/step - loss: 278.0102 - val_loss: 227.4923
Epoch 10/50
31/31 [==============================] - 0s 14ms/step - loss: 272.0822 - val_loss: 224.7529
Epoch 11/50
31/31 [==============================] - 0s 14ms/step - loss: 266.1814 - val_loss: 217.6542
Epoch 12/50
31/31 [==============================] - 0s 14ms/step - loss: 259.3674 - val_loss: 221.1896
Epoch 13/50
31/31 [==============================] - 0s 14ms/step - loss: 254.1880 - val_loss: 205.8138
Epoch 14/50
31/31 [==============================] - 0s 14ms/step - loss: 247.8201 - val_loss: 201.9908
Epoch 15/50
31/31 [==============================] - 0s 16ms/step - loss: 241.2334 - val_loss: 193.9869
Epoch 16/50
31/31 [==============================] - 0s 16ms/step - loss: 236.

31/31 [==============================] - 0s 16ms/step - loss: 258.6688 - val_loss: 210.9382
Epoch 14/50
31/31 [==============================] - 0s 16ms/step - loss: 251.7816 - val_loss: 206.8423
Epoch 15/50
31/31 [==============================] - 0s 16ms/step - loss: 244.7062 - val_loss: 198.5472
Epoch 16/50
31/31 [==============================] - 0s 16ms/step - loss: 238.3958 - val_loss: 194.2817
Epoch 17/50
31/31 [==============================] - 0s 15ms/step - loss: 232.7249 - val_loss: 190.4031
Epoch 18/50
31/31 [==============================] - 0s 15ms/step - loss: 226.5275 - val_loss: 185.9221
Epoch 19/50
31/31 [==============================] - 0s 15ms/step - loss: 222.1019 - val_loss: 185.3209
Epoch 20/50
31/31 [==============================] - 0s 15ms/step - loss: 218.6946 - val_loss: 181.7853
Epoch 21/50
31/31 [==============================] - 0s 15ms/step - loss: 215.3255 - val_loss: 178.3502
Epoch 22/50
31/31 [==============================] - 0s 15ms/step - loss: 21

47/47 [==============================] - 1s 15ms/step - loss: 204.1681 - val_loss: 359.5146
Epoch 20/50
47/47 [==============================] - 1s 14ms/step - loss: 204.2470 - val_loss: 173.7007
Epoch 21/50
47/47 [==============================] - 1s 14ms/step - loss: 200.1385 - val_loss: 199.0689
Epoch 22/50
47/47 [==============================] - 1s 14ms/step - loss: 197.5139 - val_loss: 176.0417
Epoch 23/50
47/47 [==============================] - 1s 15ms/step - loss: 195.6515 - val_loss: 196.2988
Epoch 24/50
47/47 [==============================] - 1s 15ms/step - loss: 193.4265 - val_loss: 161.8384
Epoch 25/50
47/47 [==============================] - 1s 14ms/step - loss: 189.3353 - val_loss: 162.3589
Epoch 26/50
47/47 [==============================] - 1s 14ms/step - loss: 186.0727 - val_loss: 199.5569
Epoch 27/50
47/47 [==============================] - 1s 14ms/step - loss: 184.7844 - val_loss: 232.7626
Epoch 28/50
47/47 [==============================] - 1s 15ms/step - loss: 18

Best: -167.510696 using {'optimizer': 'Adam'}

-177.097997 (5.999039) with: {'optimizer': 'SGD'}
-179.632929 (2.974254) with: {'optimizer': 'RMSprop'}
-167.510696 (5.824814) with: {'optimizer': 'Adam'}

2) Learning Rate

In [ ]:
#grid search learning rate
# create model
model = KerasRegressor(model=wider_larger_model, verbose=2, epochs = 50, batch_size = 2000, optimizer = "Adam", loss = 
                      "mean_squared_error")

# fix random seed for reproducibility
seed = 14
tf.random.set_seed(seed)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]

param_grid = dict(optimizer__learning_rate = learn_rate)#, optimizer__momentum = momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv= cross_validation, scoring = "neg_mean_squared_error")
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -168.125493 using {'optimizer__learning_rate': 0.1}

-178.255341 (4.812239) with: {'optimizer__learning_rate': 0.001}
-179.195775 (3.149702) with: {'optimizer__learning_rate': 0.01}
-168.125493 (5.164548) with: {'optimizer__learning_rate': 0.1}

3) Weight Initialization / Kernel Initializer

In [26]:
#create a combination of the wider and larger model
def wider_larger_model():
    clear_session()
    model = Sequential()
    del model
    model = Sequential()
    
    model.add(Dense(40, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
    model.add(Dense(60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(50, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(20, kernel_initializer='normal', activation='relu'))
    model.add(Dense(10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    # Compile model
    model.compile(loss='mean_squared_error')
    return model

In [27]:
#grid search network weight initialization - normal 
# Function to create model, required for KerasClassifier
# create model
model = KerasRegressor(model=wider_larger_model, verbose=2, epochs = 50, batch_size = 2000, 
                       loss = 'mean_squared_error', optimizer__learning_rate = 0.1)

# fix random seed for reproducibility
seed = 14
tf.random.set_seed(seed)

# define the grid search parameters
optimizer = ["Adam"]

param_grid = dict(optimizer = optimizer)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=cross_validation, scoring = "neg_mean_squared_error")
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
31/31 - 2s - loss: 374.6258 - 2s/epoch - 51ms/step
Epoch 2/50
31/31 - 0s - loss: 327.2076 - 371ms/epoch - 12ms/step
Epoch 3/50
31/31 - 0s - loss: 313.4785 - 356ms/epoch - 11ms/step
Epoch 4/50
31/31 - 0s - loss: 306.1486 - 313ms/epoch - 10ms/step
Epoch 5/50
31/31 - 0s - loss: 301.4651 - 351ms/epoch - 11ms/step
Epoch 6/50
31/31 - 0s - loss: 297.3497 - 357ms/epoch - 12ms/step
Epoch 7/50
31/31 - 0s - loss: 292.0669 - 344ms/epoch - 11ms/step
Epoch 8/50
31/31 - 0s - loss: 285.5938 - 372ms/epoch - 12ms/step
Epoch 9/50
31/31 - 0s - loss: 279.0816 - 374ms/epoch - 12ms/step
Epoch 10/50
31/31 - 0s - loss: 271.8571 - 356ms/epoch - 11ms/step
Epoch 11/50
31/31 - 0s - loss: 265.2194 - 352ms/epoch - 11ms/step
Epoch 12/50
31/31 - 0s - loss: 257.8468 - 335ms/epoch - 11ms/step
Epoch 13/50
31/31 - 0s - loss: 253.2977 - 346ms/epoch - 11ms/step
Epoch 14/50
31/31 - 0s - loss: 247.4047 - 335ms/epoch - 11ms/step
Epoch 15/50
31/31 - 0s - loss: 241.3210 - 321ms/epoch - 10ms/step
Epoch 16/50
31/31 - 0s

Epoch 25/50
31/31 - 0s - loss: 209.3583 - 341ms/epoch - 11ms/step
Epoch 26/50
31/31 - 0s - loss: 207.0242 - 350ms/epoch - 11ms/step
Epoch 27/50
31/31 - 0s - loss: 205.2165 - 331ms/epoch - 11ms/step
Epoch 28/50
31/31 - 0s - loss: 205.3410 - 351ms/epoch - 11ms/step
Epoch 29/50
31/31 - 0s - loss: 203.4347 - 369ms/epoch - 12ms/step
Epoch 30/50
31/31 - 0s - loss: 200.3139 - 367ms/epoch - 12ms/step
Epoch 31/50
31/31 - 0s - loss: 199.4807 - 361ms/epoch - 12ms/step
Epoch 32/50
31/31 - 0s - loss: 198.8343 - 351ms/epoch - 11ms/step
Epoch 33/50
31/31 - 0s - loss: 197.8069 - 356ms/epoch - 11ms/step
Epoch 34/50
31/31 - 0s - loss: 195.9464 - 386ms/epoch - 12ms/step
Epoch 35/50
31/31 - 0s - loss: 195.9819 - 394ms/epoch - 13ms/step
Epoch 36/50
31/31 - 0s - loss: 195.1358 - 384ms/epoch - 12ms/step
Epoch 37/50
31/31 - 0s - loss: 192.4099 - 380ms/epoch - 12ms/step
Epoch 38/50
31/31 - 0s - loss: 195.2305 - 395ms/epoch - 13ms/step
Epoch 39/50
31/31 - 0s - loss: 190.5279 - 407ms/epoch - 13ms/step
Epoch 40/5

Best: -176.601242 using {'optimizer': 'Adam'}

-176.601242 (5.547502) with: {'optimizer': 'Adam'}

In [28]:
#recreate the larger model with the uniform kernel initializer
#create a combination of the wider and larger model
def wider_larger_model():
    clear_session()
    model = Sequential()
    del model 
    model = Sequential()
    
    model.add(Dense(40, input_shape=(X_train.shape[1],), kernel_initializer='uniform', activation='relu'))
    model.add(Dense(60, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(50, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(30, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(20, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(10, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='relu'))
    # Compile model
    model.compile(loss='mean_squared_error')
    return model

In [30]:
#grid search network weight initialization - uniform
# Function to create model, required for KerasClassifier
# create model
model = KerasRegressor(model=wider_larger_model, verbose=2, epochs = 50, batch_size = 2000,
                    optimizer__learning_rate = 0.1 , loss = 'mean_squared_error')

# fix random seed for reproducibility
seed = 14
tf.random.set_seed(seed)

# define the grid search parameters
optimizer = ["Adam"]

param_grid = dict(optimizer = optimizer)


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv= cross_validation, scoring = "neg_mean_squared_error")
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
31/31 - 2s - loss: 368.9190 - 2s/epoch - 51ms/step
Epoch 2/50
31/31 - 0s - loss: 318.4619 - 330ms/epoch - 11ms/step
Epoch 3/50
31/31 - 0s - loss: 308.5445 - 328ms/epoch - 11ms/step
Epoch 4/50
31/31 - 0s - loss: 302.2253 - 325ms/epoch - 10ms/step
Epoch 5/50
31/31 - 0s - loss: 297.5529 - 322ms/epoch - 10ms/step
Epoch 6/50
31/31 - 0s - loss: 293.4004 - 310ms/epoch - 10ms/step
Epoch 7/50
31/31 - 0s - loss: 288.5132 - 314ms/epoch - 10ms/step
Epoch 8/50
31/31 - 0s - loss: 282.9726 - 342ms/epoch - 11ms/step
Epoch 9/50
31/31 - 0s - loss: 278.0172 - 321ms/epoch - 10ms/step
Epoch 10/50
31/31 - 0s - loss: 272.3779 - 371ms/epoch - 12ms/step
Epoch 11/50
31/31 - 0s - loss: 266.8022 - 351ms/epoch - 11ms/step
Epoch 12/50
31/31 - 0s - loss: 260.2900 - 327ms/epoch - 11ms/step
Epoch 13/50
31/31 - 0s - loss: 255.7745 - 340ms/epoch - 11ms/step
Epoch 14/50
31/31 - 0s - loss: 249.3157 - 340ms/epoch - 11ms/step
Epoch 15/50
31/31 - 0s - loss: 242.7481 - 330ms/epoch - 11ms/step
Epoch 16/50
31/31 - 0s

Epoch 25/50
31/31 - 0s - loss: 215.9550 - 388ms/epoch - 13ms/step
Epoch 26/50
31/31 - 0s - loss: 212.3915 - 390ms/epoch - 13ms/step
Epoch 27/50
31/31 - 0s - loss: 209.0867 - 377ms/epoch - 12ms/step
Epoch 28/50
31/31 - 0s - loss: 207.8807 - 372ms/epoch - 12ms/step
Epoch 29/50
31/31 - 0s - loss: 204.3097 - 373ms/epoch - 12ms/step
Epoch 30/50
31/31 - 0s - loss: 200.9948 - 375ms/epoch - 12ms/step
Epoch 31/50
31/31 - 0s - loss: 198.7708 - 367ms/epoch - 12ms/step
Epoch 32/50
31/31 - 0s - loss: 195.7350 - 377ms/epoch - 12ms/step
Epoch 33/50
31/31 - 0s - loss: 193.9134 - 355ms/epoch - 11ms/step
Epoch 34/50
31/31 - 0s - loss: 192.2954 - 351ms/epoch - 11ms/step
Epoch 35/50
31/31 - 0s - loss: 189.6866 - 341ms/epoch - 11ms/step
Epoch 36/50
31/31 - 0s - loss: 187.8966 - 353ms/epoch - 11ms/step
Epoch 37/50
31/31 - 0s - loss: 184.8153 - 365ms/epoch - 12ms/step
Epoch 38/50
31/31 - 0s - loss: 186.9512 - 351ms/epoch - 11ms/step
Epoch 39/50
31/31 - 0s - loss: 182.4188 - 363ms/epoch - 12ms/step
Epoch 40/5

Best: -179.578583 using {'optimizer': 'Adam'}

-179.578583 (8.723045) with: {'optimizer': 'Adam'}

3) Activation Function

In [32]:
#create a combination of the wider and larger model
def wider_larger_model():
    clear_session()
    model = Sequential()
    del model
    model = Sequential()
    
    model.add(Dense(40, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
    model.add(Dense(60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(50, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(20, kernel_initializer='normal', activation='relu'))
    model.add(Dense(10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    # Compile model
    model.compile(loss='mean_squared_error')
    return model

In [34]:
# Grid search for activation function
# create model
model = KerasRegressor(model=wider_larger_model, verbose=2, epochs = 50, batch_size = 2000, optimizer = "Adam",
                      optimizer__learning_rate = 0.1, loss = "mean_squared_error")

# fix random seed for reproducibility
seed = 14
tf.random.set_seed(seed)

# define the grid search parameters
optimizer = ["Adam"]

param_grid = dict(optimizer = optimizer)

grid = GridSearchCV(estimator=model, param_grid = param_grid, n_jobs=1, cv= cross_validation, scoring = "neg_mean_squared_error")
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
31/31 - 2s - loss: 374.6258 - 2s/epoch - 58ms/step
Epoch 2/50
31/31 - 0s - loss: 327.2076 - 291ms/epoch - 9ms/step
Epoch 3/50
31/31 - 0s - loss: 313.4785 - 320ms/epoch - 10ms/step
Epoch 4/50
31/31 - 0s - loss: 306.1486 - 319ms/epoch - 10ms/step
Epoch 5/50
31/31 - 0s - loss: 301.4651 - 317ms/epoch - 10ms/step
Epoch 6/50
31/31 - 0s - loss: 297.3497 - 316ms/epoch - 10ms/step
Epoch 7/50
31/31 - 0s - loss: 292.0669 - 339ms/epoch - 11ms/step
Epoch 8/50
31/31 - 0s - loss: 285.5938 - 316ms/epoch - 10ms/step
Epoch 9/50
31/31 - 0s - loss: 279.0816 - 300ms/epoch - 10ms/step
Epoch 10/50
31/31 - 0s - loss: 271.8571 - 328ms/epoch - 11ms/step
Epoch 11/50
31/31 - 0s - loss: 265.2194 - 321ms/epoch - 10ms/step
Epoch 12/50
31/31 - 0s - loss: 257.8468 - 304ms/epoch - 10ms/step
Epoch 13/50
31/31 - 0s - loss: 253.2977 - 339ms/epoch - 11ms/step
Epoch 14/50
31/31 - 0s - loss: 247.4047 - 303ms/epoch - 10ms/step
Epoch 15/50
31/31 - 0s - loss: 241.3210 - 312ms/epoch - 10ms/step
Epoch 16/50
31/31 - 0s 

Epoch 25/50
31/31 - 0s - loss: 209.4040 - 333ms/epoch - 11ms/step
Epoch 26/50
31/31 - 0s - loss: 207.0657 - 332ms/epoch - 11ms/step
Epoch 27/50
31/31 - 0s - loss: 205.2638 - 352ms/epoch - 11ms/step
Epoch 28/50
31/31 - 0s - loss: 205.4261 - 392ms/epoch - 13ms/step
Epoch 29/50
31/31 - 0s - loss: 203.5338 - 381ms/epoch - 12ms/step
Epoch 30/50
31/31 - 0s - loss: 200.4088 - 369ms/epoch - 12ms/step
Epoch 31/50
31/31 - 0s - loss: 199.5891 - 368ms/epoch - 12ms/step
Epoch 32/50
31/31 - 0s - loss: 198.9090 - 384ms/epoch - 12ms/step
Epoch 33/50
31/31 - 0s - loss: 197.8762 - 391ms/epoch - 13ms/step
Epoch 34/50
31/31 - 0s - loss: 196.0146 - 343ms/epoch - 11ms/step
Epoch 35/50
31/31 - 0s - loss: 195.9311 - 361ms/epoch - 12ms/step
Epoch 36/50
31/31 - 0s - loss: 195.1623 - 405ms/epoch - 13ms/step
Epoch 37/50
31/31 - 0s - loss: 192.5596 - 352ms/epoch - 11ms/step
Epoch 38/50
31/31 - 0s - loss: 195.4007 - 350ms/epoch - 11ms/step
Epoch 39/50
31/31 - 0s - loss: 190.6261 - 346ms/epoch - 11ms/step
Epoch 40/5

Best: -177.176387 using {'optimizer': 'Adam'}

-177.176387 (5.459339) with: {'optimizer': 'Adam'}

In [35]:
#create a larger model with linear activation function
def wider_larger_model():
    clear_session()
    model = Sequential()
    del model
    model = Sequential()
    
    model.add(Dense(40, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='linear'))
    model.add(Dense(60, kernel_initializer='normal', activation='linear'))
    model.add(Dense(50, kernel_initializer='normal', activation='linear'))
    model.add(Dense(30, kernel_initializer='normal', activation='linear'))
    model.add(Dense(20, kernel_initializer='normal', activation='linear'))
    model.add(Dense(10, kernel_initializer='normal', activation='linear'))
    model.add(Dense(1, kernel_initializer='normal', activation='linear'))
    # Compile model
    model.compile(loss='mean_squared_error')
    return model


In [36]:
# Grid search for activation function
# create model
model = KerasRegressor(model=wider_larger_model, verbose=2, epochs = 50, batch_size = 2000, optimizer = "Adam",
                      optimizer__learning_rate = 0.1, loss = "mean_squared_error")

# fix random seed for reproducibility
seed = 14
tf.random.set_seed(seed)

# define the grid search parameters
optimizer = ["Adam"]
param_grid = dict(optimizer = optimizer)

grid = GridSearchCV(estimator=model, param_grid = param_grid, n_jobs=1, cv= cross_validation, scoring = "neg_mean_squared_error")
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
31/31 - 1s - loss: 354.5883 - 1s/epoch - 43ms/step
Epoch 2/50
31/31 - 0s - loss: 319.0161 - 318ms/epoch - 10ms/step
Epoch 3/50
31/31 - 0s - loss: 316.6811 - 319ms/epoch - 10ms/step
Epoch 4/50
31/31 - 0s - loss: 315.4949 - 311ms/epoch - 10ms/step
Epoch 5/50
31/31 - 0s - loss: 315.0955 - 311ms/epoch - 10ms/step
Epoch 6/50
31/31 - 0s - loss: 314.9197 - 329ms/epoch - 11ms/step
Epoch 7/50
31/31 - 0s - loss: 314.4628 - 317ms/epoch - 10ms/step
Epoch 8/50
31/31 - 0s - loss: 314.1580 - 313ms/epoch - 10ms/step
Epoch 9/50
31/31 - 0s - loss: 314.1506 - 308ms/epoch - 10ms/step
Epoch 10/50
31/31 - 0s - loss: 313.9933 - 312ms/epoch - 10ms/step
Epoch 11/50
31/31 - 0s - loss: 313.9950 - 328ms/epoch - 11ms/step
Epoch 12/50
31/31 - 0s - loss: 313.7186 - 335ms/epoch - 11ms/step
Epoch 13/50
31/31 - 0s - loss: 314.1623 - 316ms/epoch - 10ms/step
Epoch 14/50
31/31 - 0s - loss: 313.8360 - 324ms/epoch - 10ms/step
Epoch 15/50
31/31 - 0s - loss: 313.6698 - 324ms/epoch - 10ms/step
Epoch 16/50
31/31 - 0s

Epoch 25/50
31/31 - 0s - loss: 305.0423 - 340ms/epoch - 11ms/step
Epoch 26/50
31/31 - 0s - loss: 305.4826 - 313ms/epoch - 10ms/step
Epoch 27/50
31/31 - 0s - loss: 305.1852 - 319ms/epoch - 10ms/step
Epoch 28/50
31/31 - 0s - loss: 305.6001 - 308ms/epoch - 10ms/step
Epoch 29/50
31/31 - 0s - loss: 305.1985 - 304ms/epoch - 10ms/step
Epoch 30/50
31/31 - 0s - loss: 305.2859 - 312ms/epoch - 10ms/step
Epoch 31/50
31/31 - 0s - loss: 305.3621 - 330ms/epoch - 11ms/step
Epoch 32/50
31/31 - 0s - loss: 305.2656 - 311ms/epoch - 10ms/step
Epoch 33/50
31/31 - 0s - loss: 305.4154 - 299ms/epoch - 10ms/step
Epoch 34/50
31/31 - 0s - loss: 305.2548 - 301ms/epoch - 10ms/step
Epoch 35/50
31/31 - 0s - loss: 305.3897 - 315ms/epoch - 10ms/step
Epoch 36/50
31/31 - 0s - loss: 305.2666 - 313ms/epoch - 10ms/step
Epoch 37/50
31/31 - 0s - loss: 305.2137 - 307ms/epoch - 10ms/step
Epoch 38/50
31/31 - 0s - loss: 305.3994 - 319ms/epoch - 10ms/step
Epoch 39/50
31/31 - 0s - loss: 305.1932 - 308ms/epoch - 10ms/step
Epoch 40/5

Best: -308.840424 using {'optimizer': 'Adam'}

-308.840424 (7.147339) with: {'optimizer': 'Adam'}

In [37]:
#create a larger model with softplus activation function 
def wider_larger_model():
    clear_session()
    model = Sequential()
    del model
    model = Sequential()
    
    model.add(Dense(40, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='softplus'))
    model.add(Dense(60, kernel_initializer='normal', activation='softplus'))
    model.add(Dense(50, kernel_initializer='normal', activation='softplus'))
    model.add(Dense(30, kernel_initializer='normal', activation='softplus'))
    model.add(Dense(20, kernel_initializer='normal', activation='softplus'))
    model.add(Dense(10, kernel_initializer='normal', activation='softplus'))
    model.add(Dense(1, kernel_initializer='normal', activation='softplus'))
    # Compile model
    model.compile(loss='mean_squared_error')
    return model


In [38]:
# Grid search for activation function
# create model
model = KerasRegressor(model=wider_larger_model, verbose=2, epochs = 50, batch_size = 2000, optimizer = "Adam",
                      optimizer__learning_rate = 0.1, loss = "mean_squared_error")

# fix random seed for reproducibility
seed = 14
tf.random.set_seed(seed)

# define the grid search parameters
optimizer = ["Adam"]
param_grid = dict(optimizer = optimizer)

grid = GridSearchCV(estimator=model, param_grid = param_grid, n_jobs=1, cv=cross_validation, scoring = "neg_mean_squared_error")
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
31/31 - 1s - loss: 367.1649 - 1s/epoch - 47ms/step
Epoch 2/50
31/31 - 0s - loss: 347.4442 - 369ms/epoch - 12ms/step
Epoch 3/50
31/31 - 0s - loss: 339.9449 - 376ms/epoch - 12ms/step
Epoch 4/50
31/31 - 0s - loss: 332.9621 - 365ms/epoch - 12ms/step
Epoch 5/50
31/31 - 0s - loss: 327.4395 - 371ms/epoch - 12ms/step
Epoch 6/50
31/31 - 0s - loss: 322.9887 - 386ms/epoch - 12ms/step
Epoch 7/50
31/31 - 0s - loss: 318.6793 - 376ms/epoch - 12ms/step
Epoch 8/50
31/31 - 0s - loss: 314.8184 - 376ms/epoch - 12ms/step
Epoch 9/50
31/31 - 0s - loss: 311.5206 - 366ms/epoch - 12ms/step
Epoch 10/50
31/31 - 0s - loss: 308.5477 - 369ms/epoch - 12ms/step
Epoch 11/50
31/31 - 0s - loss: 305.7621 - 381ms/epoch - 12ms/step
Epoch 12/50
31/31 - 0s - loss: 302.7414 - 368ms/epoch - 12ms/step
Epoch 13/50
31/31 - 0s - loss: 300.6198 - 371ms/epoch - 12ms/step
Epoch 14/50
31/31 - 0s - loss: 298.4220 - 377ms/epoch - 12ms/step
Epoch 15/50
31/31 - 0s - loss: 295.8983 - 374ms/epoch - 12ms/step
Epoch 16/50
31/31 - 0s

Epoch 25/50
31/31 - 0s - loss: 278.0937 - 391ms/epoch - 13ms/step
Epoch 26/50
31/31 - 0s - loss: 277.3705 - 371ms/epoch - 12ms/step
Epoch 27/50
31/31 - 0s - loss: 276.6261 - 372ms/epoch - 12ms/step
Epoch 28/50
31/31 - 0s - loss: 277.5193 - 372ms/epoch - 12ms/step
Epoch 29/50
31/31 - 0s - loss: 275.4961 - 368ms/epoch - 12ms/step
Epoch 30/50
31/31 - 0s - loss: 275.2768 - 372ms/epoch - 12ms/step
Epoch 31/50
31/31 - 0s - loss: 274.6918 - 405ms/epoch - 13ms/step
Epoch 32/50
31/31 - 0s - loss: 273.7508 - 342ms/epoch - 11ms/step
Epoch 33/50
31/31 - 0s - loss: 273.3243 - 317ms/epoch - 10ms/step
Epoch 34/50
31/31 - 0s - loss: 272.5322 - 330ms/epoch - 11ms/step
Epoch 35/50
31/31 - 0s - loss: 272.1854 - 336ms/epoch - 11ms/step
Epoch 36/50
31/31 - 0s - loss: 271.1839 - 337ms/epoch - 11ms/step
Epoch 37/50
31/31 - 0s - loss: 269.3404 - 363ms/epoch - 12ms/step
Epoch 38/50
31/31 - 0s - loss: 269.7838 - 349ms/epoch - 11ms/step
Epoch 39/50
31/31 - 0s - loss: 268.1416 - 343ms/epoch - 11ms/step
Epoch 40/5

Best: -251.176097 using {'optimizer': 'Adam'}

-251.176097 (10.222854) with: {'optimizer': 'Adam'}

In [42]:
#recreate the larger model without the optimizer
#create a combination of the wider and larger model
def wider_larger_model():
    clear_session()
    model = Sequential()
    del model
    model = Sequential()
    
    model.add(Dense(40, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
    model.add(Dense(60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(50, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(20, kernel_initializer='normal', activation='relu'))
    model.add(Dense(10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='linear'))
    # Compile model
    model.compile(loss='mean_squared_error')
    return model

In [40]:
# Grid search optimizer
# Function to create model, required for KerasClassifier
model = KerasRegressor(model=wider_larger_model, verbose=2, epochs = 50, batch_size = 2000, optimizer = "Adam",
                      optimizer__learning_rate = 0.1, loss = "mean_squared_error")

# fix random seed for reproducibility
seed = 14
tf.random.set_seed(seed)

# define the grid search parameters
optimizer = ["Adam"]
param_grid = dict(optimizer = optimizer)

grid = GridSearchCV(estimator=model, param_grid = param_grid, n_jobs=1, cv= cross_validation, scoring = "neg_mean_squared_error")
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
31/31 - 2s - loss: 374.3952 - 2s/epoch - 69ms/step
Epoch 2/50
31/31 - 0s - loss: 326.7945 - 355ms/epoch - 11ms/step
Epoch 3/50
31/31 - 0s - loss: 313.2644 - 378ms/epoch - 12ms/step
Epoch 4/50
31/31 - 0s - loss: 305.9855 - 380ms/epoch - 12ms/step
Epoch 5/50
31/31 - 0s - loss: 301.3784 - 409ms/epoch - 13ms/step
Epoch 6/50
31/31 - 0s - loss: 297.4115 - 409ms/epoch - 13ms/step
Epoch 7/50
31/31 - 0s - loss: 292.4056 - 380ms/epoch - 12ms/step
Epoch 8/50
31/31 - 0s - loss: 286.2470 - 376ms/epoch - 12ms/step
Epoch 9/50
31/31 - 0s - loss: 279.9827 - 387ms/epoch - 12ms/step
Epoch 10/50
31/31 - 0s - loss: 272.9116 - 387ms/epoch - 12ms/step
Epoch 11/50
31/31 - 0s - loss: 266.3144 - 382ms/epoch - 12ms/step
Epoch 12/50
31/31 - 0s - loss: 259.0343 - 409ms/epoch - 13ms/step
Epoch 13/50
31/31 - 0s - loss: 254.5285 - 405ms/epoch - 13ms/step
Epoch 14/50
31/31 - 0s - loss: 248.5059 - 413ms/epoch - 13ms/step
Epoch 15/50
31/31 - 0s - loss: 242.4583 - 423ms/epoch - 14ms/step
Epoch 16/50
31/31 - 0s

Epoch 25/50
31/31 - 0s - loss: 208.8521 - 424ms/epoch - 14ms/step
Epoch 26/50
31/31 - 0s - loss: 204.9390 - 415ms/epoch - 13ms/step
Epoch 27/50
31/31 - 0s - loss: 203.9034 - 415ms/epoch - 13ms/step
Epoch 28/50
31/31 - 0s - loss: 203.2296 - 391ms/epoch - 13ms/step
Epoch 29/50
31/31 - 0s - loss: 200.3141 - 351ms/epoch - 11ms/step
Epoch 30/50
31/31 - 0s - loss: 197.0702 - 357ms/epoch - 12ms/step
Epoch 31/50
31/31 - 0s - loss: 195.0454 - 383ms/epoch - 12ms/step
Epoch 32/50
31/31 - 0s - loss: 194.0886 - 375ms/epoch - 12ms/step
Epoch 33/50
31/31 - 0s - loss: 191.8759 - 349ms/epoch - 11ms/step
Epoch 34/50
31/31 - 0s - loss: 189.5077 - 359ms/epoch - 12ms/step
Epoch 35/50
31/31 - 0s - loss: 188.4102 - 358ms/epoch - 12ms/step
Epoch 36/50
31/31 - 0s - loss: 186.9196 - 362ms/epoch - 12ms/step
Epoch 37/50
31/31 - 0s - loss: 184.5864 - 352ms/epoch - 11ms/step
Epoch 38/50
31/31 - 0s - loss: 186.4511 - 357ms/epoch - 12ms/step
Epoch 39/50
31/31 - 0s - loss: 181.1065 - 346ms/epoch - 11ms/step
Epoch 40/5

Best: -182.506612 using {'optimizer': 'Adam'}

-182.506612 (13.869358) with: {'optimizer': 'Adam'}

In [15]:
#taking the best model - with relu activation function
def wider_larger_model():
    clear_session()
    model = Sequential()
    del model
    model = Sequential()
    
    model.add(Dense(40, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
    model.add(Dense(60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(50, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(20, kernel_initializer='normal', activation='relu'))
    model.add(Dense(10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    # Compile model
    model.compile(loss='mean_squared_error')
    return model

In [16]:
##increasing epochs to 100

model = KerasRegressor(model=wider_larger_model, verbose=2, epochs = 100, batch_size = 2000, optimizer = "Adam",
                      optimizer__learning_rate = 0.1, loss = "mean_squared_error")

# fix random seed for reproducibility
seed = 14
tf.random.set_seed(seed)

# define the grid search parameters
optimizer = ["Adam"]
param_grid = dict(optimizer = optimizer)

grid = GridSearchCV(estimator=model, param_grid = param_grid, n_jobs=1, cv= cross_validation, scoring = "neg_mean_squared_error")
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/100
31/31 - 3s - loss: 374.6258 - 3s/epoch - 100ms/step
Epoch 2/100
31/31 - 0s - loss: 327.2076 - 319ms/epoch - 10ms/step
Epoch 3/100
31/31 - 0s - loss: 313.4785 - 331ms/epoch - 11ms/step
Epoch 4/100
31/31 - 0s - loss: 306.1486 - 314ms/epoch - 10ms/step
Epoch 5/100
31/31 - 0s - loss: 301.4651 - 323ms/epoch - 10ms/step
Epoch 6/100
31/31 - 0s - loss: 297.3497 - 321ms/epoch - 10ms/step
Epoch 7/100
31/31 - 0s - loss: 292.0669 - 325ms/epoch - 10ms/step
Epoch 8/100
31/31 - 0s - loss: 285.5938 - 334ms/epoch - 11ms/step
Epoch 9/100
31/31 - 0s - loss: 279.0816 - 341ms/epoch - 11ms/step
Epoch 10/100
31/31 - 0s - loss: 271.8571 - 334ms/epoch - 11ms/step
Epoch 11/100
31/31 - 0s - loss: 265.2194 - 322ms/epoch - 10ms/step
Epoch 12/100
31/31 - 0s - loss: 257.8468 - 335ms/epoch - 11ms/step
Epoch 13/100
31/31 - 0s - loss: 253.2977 - 326ms/epoch - 11ms/step
Epoch 14/100
31/31 - 0s - loss: 247.4047 - 326ms/epoch - 11ms/step
Epoch 15/100
31/31 - 0s - loss: 241.3210 - 337ms/epoch - 11ms/step
Epoch 

31/31 - 0s - loss: 210.9435 - 368ms/epoch - 12ms/step
Epoch 24/100
31/31 - 0s - loss: 208.7715 - 355ms/epoch - 11ms/step
Epoch 25/100
31/31 - 0s - loss: 206.4456 - 322ms/epoch - 10ms/step
Epoch 26/100
31/31 - 0s - loss: 204.2942 - 329ms/epoch - 11ms/step
Epoch 27/100
31/31 - 0s - loss: 201.4770 - 355ms/epoch - 11ms/step
Epoch 28/100
31/31 - 0s - loss: 198.6564 - 348ms/epoch - 11ms/step
Epoch 29/100
31/31 - 0s - loss: 196.5705 - 343ms/epoch - 11ms/step
Epoch 30/100
31/31 - 0s - loss: 194.2440 - 349ms/epoch - 11ms/step
Epoch 31/100
31/31 - 0s - loss: 190.6325 - 351ms/epoch - 11ms/step
Epoch 32/100
31/31 - 0s - loss: 189.6891 - 335ms/epoch - 11ms/step
Epoch 33/100
31/31 - 0s - loss: 186.1621 - 342ms/epoch - 11ms/step
Epoch 34/100
31/31 - 0s - loss: 187.3866 - 352ms/epoch - 11ms/step
Epoch 35/100
31/31 - 0s - loss: 186.0957 - 341ms/epoch - 11ms/step
Epoch 36/100
31/31 - 0s - loss: 180.7451 - 330ms/epoch - 11ms/step
Epoch 37/100
31/31 - 0s - loss: 180.8879 - 322ms/epoch - 10ms/step
Epoch 38

31/31 - 0s - loss: 187.7290 - 354ms/epoch - 11ms/step
Epoch 46/100
31/31 - 0s - loss: 185.3327 - 366ms/epoch - 12ms/step
Epoch 47/100
31/31 - 0s - loss: 183.3572 - 372ms/epoch - 12ms/step
Epoch 48/100
31/31 - 0s - loss: 183.3616 - 385ms/epoch - 12ms/step
Epoch 49/100
31/31 - 0s - loss: 183.1560 - 367ms/epoch - 12ms/step
Epoch 50/100
31/31 - 0s - loss: 181.6232 - 360ms/epoch - 12ms/step
Epoch 51/100
31/31 - 0s - loss: 180.4631 - 364ms/epoch - 12ms/step
Epoch 52/100
31/31 - 0s - loss: 179.2064 - 355ms/epoch - 11ms/step
Epoch 53/100
31/31 - 0s - loss: 180.0383 - 354ms/epoch - 11ms/step
Epoch 54/100
31/31 - 0s - loss: 178.0523 - 368ms/epoch - 12ms/step
Epoch 55/100
31/31 - 0s - loss: 178.8456 - 359ms/epoch - 12ms/step
Epoch 56/100
31/31 - 0s - loss: 175.7813 - 377ms/epoch - 12ms/step
Epoch 57/100
31/31 - 0s - loss: 179.3104 - 376ms/epoch - 12ms/step
Epoch 58/100
31/31 - 0s - loss: 173.7488 - 372ms/epoch - 12ms/step
Epoch 59/100
31/31 - 0s - loss: 174.1573 - 387ms/epoch - 12ms/step
Epoch 60

47/47 - 1s - loss: 143.8476 - 514ms/epoch - 11ms/step
Epoch 68/100
47/47 - 1s - loss: 144.6333 - 513ms/epoch - 11ms/step
Epoch 69/100
47/47 - 1s - loss: 141.6842 - 519ms/epoch - 11ms/step
Epoch 70/100
47/47 - 1s - loss: 144.4512 - 523ms/epoch - 11ms/step
Epoch 71/100
47/47 - 1s - loss: 141.8470 - 518ms/epoch - 11ms/step
Epoch 72/100
47/47 - 0s - loss: 143.0563 - 498ms/epoch - 11ms/step
Epoch 73/100
47/47 - 1s - loss: 141.8799 - 514ms/epoch - 11ms/step
Epoch 74/100
47/47 - 1s - loss: 140.8903 - 530ms/epoch - 11ms/step
Epoch 75/100
47/47 - 1s - loss: 148.3981 - 506ms/epoch - 11ms/step
Epoch 76/100
47/47 - 1s - loss: 142.1972 - 505ms/epoch - 11ms/step
Epoch 77/100
47/47 - 1s - loss: 140.3032 - 511ms/epoch - 11ms/step
Epoch 78/100
47/47 - 1s - loss: 138.3260 - 533ms/epoch - 11ms/step
Epoch 79/100
47/47 - 1s - loss: 136.9226 - 525ms/epoch - 11ms/step
Epoch 80/100
47/47 - 1s - loss: 136.7989 - 533ms/epoch - 11ms/step
Epoch 81/100
47/47 - 1s - loss: 139.9119 - 551ms/epoch - 12ms/step
Epoch 82

Best: -147.060699 using {'optimizer': 'Adam'}

-147.060699 (5.443836) with: {'optimizer': 'Adam'}

4) Dropout and MaxNorm

In [17]:
#add dropout layers and weight contstraints
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm

def wider_larger_model(dropout_rate, weight_constraint):
    clear_session()
    model = Sequential()
    del model
    model = Sequential()
    
    model.add(Dense(40, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
    #model.add(Dropout(dropout_rate))
    model.add(Dense(60, kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(50, kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(30, kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(20, kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(10, kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    # Compile model
    model.compile(loss='mean_squared_error')
    return model

In [18]:
#grid search dropout and weight constraint
# create model
model = KerasRegressor(model=wider_larger_model, verbose=2, epochs = 100, batch_size = 2000, optimizer = "Adam",
                      optimizer__learning_rate = 0.1, loss = 'mean_squared_error')

# define the grid search parameters
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.025, 0.05, 0.1, 0.2]
# Make a dictionary of the grid search parameters
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint = weight_constraint )


grid = GridSearchCV(estimator=model, param_grid = param_grid, n_jobs=1, cv= cross_validation, scoring = "neg_mean_squared_error")
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/100
31/31 - 2s - loss: 375.8441 - 2s/epoch - 61ms/step
Epoch 2/100
31/31 - 0s - loss: 329.0002 - 419ms/epoch - 14ms/step
Epoch 3/100
31/31 - 0s - loss: 314.0580 - 407ms/epoch - 13ms/step
Epoch 4/100
31/31 - 0s - loss: 306.6778 - 413ms/epoch - 13ms/step
Epoch 5/100
31/31 - 0s - loss: 302.0614 - 427ms/epoch - 14ms/step
Epoch 6/100
31/31 - 0s - loss: 298.3149 - 406ms/epoch - 13ms/step
Epoch 7/100
31/31 - 0s - loss: 293.9109 - 424ms/epoch - 14ms/step
Epoch 8/100
31/31 - 0s - loss: 288.9322 - 424ms/epoch - 14ms/step
Epoch 9/100
31/31 - 0s - loss: 284.2597 - 447ms/epoch - 14ms/step
Epoch 10/100
31/31 - 0s - loss: 279.2475 - 449ms/epoch - 14ms/step
Epoch 11/100
31/31 - 0s - loss: 274.5540 - 428ms/epoch - 14ms/step
Epoch 12/100
31/31 - 0s - loss: 269.0661 - 428ms/epoch - 14ms/step
Epoch 13/100
31/31 - 0s - loss: 264.9795 - 412ms/epoch - 13ms/step
Epoch 14/100
31/31 - 0s - loss: 259.4572 - 446ms/epoch - 14ms/step
Epoch 15/100
31/31 - 0s - loss: 253.7988 - 454ms/epoch - 15ms/step
Epoch 1

31/31 - 0s - loss: 206.5541 - 433ms/epoch - 14ms/step
Epoch 24/100
31/31 - 0s - loss: 205.7985 - 422ms/epoch - 14ms/step
Epoch 25/100
31/31 - 0s - loss: 203.9504 - 396ms/epoch - 13ms/step
Epoch 26/100
31/31 - 0s - loss: 202.6473 - 394ms/epoch - 13ms/step
Epoch 27/100
31/31 - 0s - loss: 200.1305 - 428ms/epoch - 14ms/step
Epoch 28/100
31/31 - 0s - loss: 197.4482 - 401ms/epoch - 13ms/step
Epoch 29/100
31/31 - 0s - loss: 198.5042 - 429ms/epoch - 14ms/step
Epoch 30/100
31/31 - 0s - loss: 194.9116 - 432ms/epoch - 14ms/step
Epoch 31/100
31/31 - 0s - loss: 194.2865 - 425ms/epoch - 14ms/step
Epoch 32/100
31/31 - 0s - loss: 193.5352 - 434ms/epoch - 14ms/step
Epoch 33/100
31/31 - 0s - loss: 191.3140 - 427ms/epoch - 14ms/step
Epoch 34/100
31/31 - 0s - loss: 192.0205 - 391ms/epoch - 13ms/step
Epoch 35/100
31/31 - 0s - loss: 190.8712 - 437ms/epoch - 14ms/step
Epoch 36/100
31/31 - 0s - loss: 188.3980 - 409ms/epoch - 13ms/step
Epoch 37/100
31/31 - 0s - loss: 188.7382 - 413ms/epoch - 13ms/step
Epoch 38

31/31 - 0s - loss: 182.4378 - 403ms/epoch - 13ms/step
Epoch 46/100
31/31 - 0s - loss: 181.6317 - 404ms/epoch - 13ms/step
Epoch 47/100
31/31 - 0s - loss: 180.2939 - 385ms/epoch - 12ms/step
Epoch 48/100
31/31 - 0s - loss: 178.5971 - 401ms/epoch - 13ms/step
Epoch 49/100
31/31 - 0s - loss: 180.2861 - 405ms/epoch - 13ms/step
Epoch 50/100
31/31 - 0s - loss: 178.7603 - 393ms/epoch - 13ms/step
Epoch 51/100
31/31 - 0s - loss: 177.9903 - 409ms/epoch - 13ms/step
Epoch 52/100
31/31 - 0s - loss: 176.4755 - 388ms/epoch - 13ms/step
Epoch 53/100
31/31 - 0s - loss: 177.6035 - 396ms/epoch - 13ms/step
Epoch 54/100
31/31 - 0s - loss: 175.5381 - 407ms/epoch - 13ms/step
Epoch 55/100
31/31 - 0s - loss: 177.0675 - 419ms/epoch - 14ms/step
Epoch 56/100
31/31 - 0s - loss: 172.8475 - 399ms/epoch - 13ms/step
Epoch 57/100
31/31 - 0s - loss: 177.0136 - 426ms/epoch - 14ms/step
Epoch 58/100
31/31 - 0s - loss: 173.3835 - 403ms/epoch - 13ms/step
Epoch 59/100
31/31 - 0s - loss: 172.1355 - 417ms/epoch - 13ms/step
Epoch 60

31/31 - 0s - loss: 144.2048 - 407ms/epoch - 13ms/step
Epoch 68/100
31/31 - 0s - loss: 142.7483 - 413ms/epoch - 13ms/step
Epoch 69/100
31/31 - 0s - loss: 145.2729 - 423ms/epoch - 14ms/step
Epoch 70/100
31/31 - 0s - loss: 143.7145 - 408ms/epoch - 13ms/step
Epoch 71/100
31/31 - 0s - loss: 139.8454 - 391ms/epoch - 13ms/step
Epoch 72/100
31/31 - 0s - loss: 140.9189 - 417ms/epoch - 13ms/step
Epoch 73/100
31/31 - 0s - loss: 140.9526 - 411ms/epoch - 13ms/step
Epoch 74/100
31/31 - 0s - loss: 140.3565 - 403ms/epoch - 13ms/step
Epoch 75/100
31/31 - 0s - loss: 140.9249 - 407ms/epoch - 13ms/step
Epoch 76/100
31/31 - 0s - loss: 140.7577 - 422ms/epoch - 14ms/step
Epoch 77/100
31/31 - 0s - loss: 141.8211 - 413ms/epoch - 13ms/step
Epoch 78/100
31/31 - 0s - loss: 138.5630 - 400ms/epoch - 13ms/step
Epoch 79/100
31/31 - 0s - loss: 139.9675 - 422ms/epoch - 14ms/step
Epoch 80/100
31/31 - 0s - loss: 139.2720 - 441ms/epoch - 14ms/step
Epoch 81/100
31/31 - 0s - loss: 137.1977 - 426ms/epoch - 14ms/step
Epoch 82

31/31 - 0s - loss: 161.6459 - 425ms/epoch - 14ms/step
Epoch 90/100
31/31 - 0s - loss: 166.4459 - 446ms/epoch - 14ms/step
Epoch 91/100
31/31 - 0s - loss: 161.9825 - 425ms/epoch - 14ms/step
Epoch 92/100
31/31 - 0s - loss: 158.9622 - 427ms/epoch - 14ms/step
Epoch 93/100
31/31 - 0s - loss: 160.1959 - 417ms/epoch - 13ms/step
Epoch 94/100
31/31 - 0s - loss: 157.8672 - 420ms/epoch - 14ms/step
Epoch 95/100
31/31 - 0s - loss: 159.9249 - 428ms/epoch - 14ms/step
Epoch 96/100
31/31 - 0s - loss: 159.4484 - 450ms/epoch - 15ms/step
Epoch 97/100
31/31 - 0s - loss: 154.9014 - 416ms/epoch - 13ms/step
Epoch 98/100
31/31 - 0s - loss: 157.4352 - 422ms/epoch - 14ms/step
Epoch 99/100
31/31 - 0s - loss: 154.8088 - 430ms/epoch - 14ms/step
Epoch 100/100
31/31 - 0s - loss: 157.1324 - 429ms/epoch - 14ms/step
16/16 - 0s - 150ms/epoch - 9ms/step
Epoch 1/100
31/31 - 2s - loss: 361.8445 - 2s/epoch - 60ms/step
Epoch 2/100
31/31 - 0s - loss: 315.8872 - 410ms/epoch - 13ms/step
Epoch 3/100
31/31 - 0s - loss: 303.3795 - 4

...

Epoch 77/100
31/31 - 0s - loss: 160.9286 - 458ms/epoch - 15ms/step
Epoch 78/100
31/31 - 0s - loss: 157.1479 - 459ms/epoch - 15ms/step
Epoch 79/100
31/31 - 0s - loss: 156.5493 - 446ms/epoch - 14ms/step
Epoch 80/100
31/31 - 0s - loss: 161.6963 - 452ms/epoch - 15ms/step
Epoch 81/100
31/31 - 0s - loss: 156.0062 - 473ms/epoch - 15ms/step
Epoch 82/100
31/31 - 0s - loss: 156.9622 - 445ms/epoch - 14ms/step
Epoch 83/100
31/31 - 0s - loss: 154.9874 - 467ms/epoch - 15ms/step
Epoch 84/100
31/31 - 0s - loss: 154.7612 - 483ms/epoch - 16ms/step
Epoch 85/100
31/31 - 0s - loss: 158.2554 - 460ms/epoch - 15ms/step
Epoch 86/100
31/31 - 0s - loss: 152.0425 - 460ms/epoch - 15ms/step
Epoch 87/100
31/31 - 0s - loss: 153.1253 - 472ms/epoch - 15ms/step
Epoch 88/100
31/31 - 1s - loss: 155.5896 - 500ms/epoch - 16ms/step
Epoch 89/100
31/31 - 0s - loss: 153.6497 - 477ms/epoch - 15ms/step
Epoch 90/100
31/31 - 0s - loss: 156.8990 - 485ms/epoch - 16ms/step
Epoch 91/100
31/31 - 0s - loss: 155.1873 - 486ms/epoch - 16ms/

Epoch 99/100
31/31 - 0s - loss: 159.8769 - 459ms/epoch - 15ms/step
Epoch 100/100
31/31 - 0s - loss: 162.0596 - 454ms/epoch - 15ms/step
16/16 - 0s - 146ms/epoch - 9ms/step
Epoch 1/100
31/31 - 2s - loss: 367.6398 - 2s/epoch - 78ms/step
Epoch 2/100
31/31 - 0s - loss: 325.4937 - 464ms/epoch - 15ms/step
Epoch 3/100
31/31 - 0s - loss: 314.4615 - 474ms/epoch - 15ms/step
Epoch 4/100
31/31 - 0s - loss: 308.9557 - 463ms/epoch - 15ms/step
Epoch 5/100
31/31 - 0s - loss: 303.1303 - 470ms/epoch - 15ms/step
Epoch 6/100
31/31 - 0s - loss: 296.4472 - 462ms/epoch - 15ms/step
Epoch 7/100
31/31 - 0s - loss: 290.5708 - 468ms/epoch - 15ms/step
Epoch 8/100
31/31 - 0s - loss: 282.3629 - 444ms/epoch - 14ms/step
Epoch 9/100
31/31 - 0s - loss: 274.4979 - 465ms/epoch - 15ms/step
Epoch 10/100
31/31 - 0s - loss: 269.1327 - 470ms/epoch - 15ms/step
Epoch 11/100
31/31 - 0s - loss: 260.1194 - 469ms/epoch - 15ms/step
Epoch 12/100
31/31 - 0s - loss: 256.8869 - 460ms/epoch - 15ms/step
Epoch 13/100
31/31 - 0s - loss: 251.6

Epoch 21/100
31/31 - 0s - loss: 230.8565 - 481ms/epoch - 16ms/step
Epoch 22/100
31/31 - 0s - loss: 226.2629 - 471ms/epoch - 15ms/step
Epoch 23/100
31/31 - 0s - loss: 219.0883 - 465ms/epoch - 15ms/step
Epoch 24/100
31/31 - 0s - loss: 220.5409 - 478ms/epoch - 15ms/step
Epoch 25/100
31/31 - 0s - loss: 218.0079 - 480ms/epoch - 15ms/step
Epoch 26/100
31/31 - 0s - loss: 215.8215 - 474ms/epoch - 15ms/step
Epoch 27/100
31/31 - 0s - loss: 213.9177 - 482ms/epoch - 16ms/step
Epoch 28/100
31/31 - 0s - loss: 210.8264 - 467ms/epoch - 15ms/step
Epoch 29/100
31/31 - 0s - loss: 208.9401 - 459ms/epoch - 15ms/step
Epoch 30/100
31/31 - 0s - loss: 206.0226 - 464ms/epoch - 15ms/step
Epoch 31/100
31/31 - 0s - loss: 205.7014 - 491ms/epoch - 16ms/step
Epoch 32/100
31/31 - 0s - loss: 203.9419 - 461ms/epoch - 15ms/step
Epoch 33/100
31/31 - 0s - loss: 197.9240 - 488ms/epoch - 16ms/step
Epoch 34/100
31/31 - 0s - loss: 201.9259 - 474ms/epoch - 15ms/step
Epoch 35/100
31/31 - 0s - loss: 197.8559 - 470ms/epoch - 15ms/

Epoch 43/100
31/31 - 0s - loss: 190.1232 - 482ms/epoch - 16ms/step
Epoch 44/100
31/31 - 0s - loss: 191.9804 - 491ms/epoch - 16ms/step
Epoch 45/100
31/31 - 0s - loss: 194.0248 - 494ms/epoch - 16ms/step
Epoch 46/100
31/31 - 0s - loss: 193.3343 - 487ms/epoch - 16ms/step
Epoch 47/100
31/31 - 1s - loss: 188.5388 - 505ms/epoch - 16ms/step
Epoch 48/100
31/31 - 0s - loss: 190.3297 - 498ms/epoch - 16ms/step
Epoch 49/100
31/31 - 0s - loss: 188.5906 - 495ms/epoch - 16ms/step
Epoch 50/100
31/31 - 0s - loss: 187.3030 - 493ms/epoch - 16ms/step
Epoch 51/100
31/31 - 0s - loss: 184.1321 - 462ms/epoch - 15ms/step
Epoch 52/100
31/31 - 0s - loss: 186.6427 - 480ms/epoch - 15ms/step
Epoch 53/100
31/31 - 0s - loss: 185.8048 - 489ms/epoch - 16ms/step
Epoch 54/100
31/31 - 0s - loss: 186.4448 - 467ms/epoch - 15ms/step
Epoch 55/100
31/31 - 0s - loss: 187.6908 - 475ms/epoch - 15ms/step
Epoch 56/100
31/31 - 0s - loss: 185.1045 - 476ms/epoch - 15ms/step
Epoch 57/100
31/31 - 0s - loss: 189.3016 - 478ms/epoch - 15ms/

Epoch 65/100
47/47 - 1s - loss: 141.4377 - 681ms/epoch - 14ms/step
Epoch 66/100
47/47 - 1s - loss: 141.2825 - 674ms/epoch - 14ms/step
Epoch 67/100
47/47 - 1s - loss: 137.0623 - 680ms/epoch - 14ms/step
Epoch 68/100
47/47 - 1s - loss: 137.4625 - 715ms/epoch - 15ms/step
Epoch 69/100
47/47 - 1s - loss: 137.7123 - 736ms/epoch - 16ms/step
Epoch 70/100
47/47 - 1s - loss: 139.1271 - 714ms/epoch - 15ms/step
Epoch 71/100
47/47 - 1s - loss: 138.3910 - 697ms/epoch - 15ms/step
Epoch 72/100
47/47 - 1s - loss: 139.3553 - 697ms/epoch - 15ms/step
Epoch 73/100
47/47 - 1s - loss: 136.4774 - 727ms/epoch - 15ms/step
Epoch 74/100
47/47 - 1s - loss: 135.5694 - 734ms/epoch - 16ms/step
Epoch 75/100
47/47 - 1s - loss: 139.2535 - 715ms/epoch - 15ms/step
Epoch 76/100
47/47 - 1s - loss: 138.3625 - 730ms/epoch - 16ms/step
Epoch 77/100
47/47 - 1s - loss: 135.8101 - 680ms/epoch - 14ms/step
Epoch 78/100
47/47 - 1s - loss: 134.7331 - 712ms/epoch - 15ms/step
Epoch 79/100
47/47 - 1s - loss: 136.0341 - 696ms/epoch - 15ms/

Best: -137.486476 using {'model__dropout_rate': 0.05, 'model__weight_constraint': 2}

-156.424255 (6.402800) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1}
-149.643926 (10.127500) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2}
-227.073634 (109.808196) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3}
-148.130417 (5.963865) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4}
-217.872808 (116.406006) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5}
-146.754669 (5.336987) with: {'model__dropout_rate': 0.025, 'model__weight_constraint': 1}
-140.456487 (2.091939) with: {'model__dropout_rate': 0.025, 'model__weight_constraint': 2}
-146.808904 (7.819126) with: {'model__dropout_rate': 0.025, 'model__weight_constraint': 3}
-141.397319 (6.629934) with: {'model__dropout_rate': 0.025, 'model__weight_constraint': 4}
-218.636470 (113.976668) with: {'model__dropout_rate': 0.025, 'model__weight_constraint': 5}
-151.123322 (1.592538) with: {'model__dropout_rate': 0.05, 'model__weight_constraint': 1}
-137.486476 (3.889243) with: {'model__dropout_rate': 0.05, 'model__weight_constraint': 2}
-140.159988 (4.910397) with: {'model__dropout_rate': 0.05, 'model__weight_constraint': 3}
-144.266953 (6.253981) with: {'model__dropout_rate': 0.05, 'model__weight_constraint': 4}
-140.873662 (4.485449) with: {'model__dropout_rate': 0.05, 'model__weight_constraint': 5}
-146.297028 (8.973780) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1}
-149.692561 (12.045683) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2}
-143.401321 (2.547403) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3}
-148.242701 (10.360102) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4}
-143.774618 (9.032373) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5}
-146.450750 (7.950338) with: {'model__dropout_rate': 0.2, 'model__weight_constraint': 1}
-149.564056 (10.605673) with: {'model__dropout_rate': 0.2, 'model__weight_constraint': 2}
-149.845011 (6.708031) with: {'model__dropout_rate': 0.2, 'model__weight_constraint': 3}
-145.286397 (5.171184) with: {'model__dropout_rate': 0.2, 'model__weight_constraint': 4}
-150.745468 (6.109678) with: {'model__dropout_rate': 0.2, 'model__weight_constraint': 5}

In [19]:
#taking the best model - with dropout and weight constraint in input layer
def wider_larger_model(dropout_rate, weight_constraint):
    clear_session()
    model = Sequential()
    del model
    model = Sequential()
    
    model.add(Dense(40, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(60, kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(2)))
    model.add(Dropout(0.05))
    model.add(Dense(50, kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(2)))
    model.add(Dropout(0.05))
    model.add(Dense(30, kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(2)))
    model.add(Dropout(0.05))
    model.add(Dense(20, kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(2)))
    model.add(Dropout(0.05))
    model.add(Dense(10, kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(2)))
    model.add(Dropout(0.05))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    # Compile model
    model.compile(loss='mean_squared_error')
    return model

In [20]:
# create model
model = KerasRegressor(model=wider_larger_model, verbose=2, epochs = 100, batch_size = 2000, optimizer = "Adam",
                      optimizer__learning_rate = 0.1, loss = 'mean_squared_error')

# define the grid search parameters
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.025, 0.05, 0.1, 0.2]
# Make a dictionary of the grid search parameters
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint = weight_constraint )


grid = GridSearchCV(estimator=model, param_grid = param_grid, n_jobs=1, cv= cross_validation, scoring = "neg_mean_squared_error")
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/100
31/31 - 2s - loss: 369.3187 - 2s/epoch - 71ms/step
Epoch 2/100
31/31 - 0s - loss: 322.6279 - 417ms/epoch - 13ms/step
Epoch 3/100
31/31 - 0s - loss: 313.0153 - 416ms/epoch - 13ms/step
Epoch 4/100
31/31 - 0s - loss: 305.5004 - 373ms/epoch - 12ms/step
Epoch 5/100
31/31 - 0s - loss: 300.2828 - 426ms/epoch - 14ms/step
Epoch 6/100
31/31 - 0s - loss: 296.2774 - 394ms/epoch - 13ms/step
Epoch 7/100
31/31 - 0s - loss: 291.8758 - 378ms/epoch - 12ms/step
Epoch 8/100
31/31 - 0s - loss: 284.5204 - 440ms/epoch - 14ms/step
Epoch 9/100
31/31 - 0s - loss: 279.2496 - 416ms/epoch - 13ms/step
Epoch 10/100
31/31 - 0s - loss: 274.3894 - 426ms/epoch - 14ms/step
Epoch 11/100
31/31 - 0s - loss: 267.8240 - 426ms/epoch - 14ms/step
Epoch 12/100
31/31 - 0s - loss: 259.6347 - 452ms/epoch - 15ms/step
Epoch 13/100
31/31 - 0s - loss: 253.5359 - 421ms/epoch - 14ms/step
Epoch 14/100
31/31 - 0s - loss: 247.2786 - 460ms/epoch - 15ms/step
Epoch 15/100
31/31 - 0s - loss: 242.7149 - 424ms/epoch - 14ms/step
Epoch 1

31/31 - 0s - loss: 213.7769 - 457ms/epoch - 15ms/step
Epoch 24/100
31/31 - 0s - loss: 212.1396 - 467ms/epoch - 15ms/step
Epoch 25/100
31/31 - 0s - loss: 209.9778 - 440ms/epoch - 14ms/step
Epoch 26/100
31/31 - 0s - loss: 207.9136 - 476ms/epoch - 15ms/step
Epoch 27/100
31/31 - 0s - loss: 204.5136 - 473ms/epoch - 15ms/step
Epoch 28/100
31/31 - 0s - loss: 202.3418 - 449ms/epoch - 14ms/step
Epoch 29/100
31/31 - 0s - loss: 199.3338 - 469ms/epoch - 15ms/step
Epoch 30/100
31/31 - 0s - loss: 198.2426 - 437ms/epoch - 14ms/step
Epoch 31/100
31/31 - 0s - loss: 196.7695 - 464ms/epoch - 15ms/step
Epoch 32/100
31/31 - 0s - loss: 192.1961 - 436ms/epoch - 14ms/step
Epoch 33/100
31/31 - 0s - loss: 190.3463 - 470ms/epoch - 15ms/step
Epoch 34/100
31/31 - 0s - loss: 190.1109 - 444ms/epoch - 14ms/step
Epoch 35/100
31/31 - 0s - loss: 187.9004 - 443ms/epoch - 14ms/step
Epoch 36/100
31/31 - 0s - loss: 186.2041 - 480ms/epoch - 15ms/step
Epoch 37/100
31/31 - 0s - loss: 184.0455 - 417ms/epoch - 13ms/step
Epoch 38

31/31 - 0s - loss: 186.3265 - 419ms/epoch - 14ms/step
Epoch 46/100
31/31 - 0s - loss: 183.0044 - 495ms/epoch - 16ms/step
Epoch 47/100
31/31 - 0s - loss: 183.7227 - 464ms/epoch - 15ms/step
Epoch 48/100
31/31 - 0s - loss: 179.3665 - 465ms/epoch - 15ms/step
Epoch 49/100
31/31 - 0s - loss: 181.5574 - 454ms/epoch - 15ms/step
Epoch 50/100
31/31 - 0s - loss: 177.9938 - 456ms/epoch - 15ms/step
Epoch 51/100
31/31 - 0s - loss: 177.8316 - 439ms/epoch - 14ms/step
Epoch 52/100
31/31 - 0s - loss: 174.6147 - 434ms/epoch - 14ms/step
Epoch 53/100
31/31 - 0s - loss: 178.8772 - 487ms/epoch - 16ms/step
Epoch 54/100
31/31 - 0s - loss: 173.8978 - 482ms/epoch - 16ms/step
Epoch 55/100
31/31 - 0s - loss: 173.8806 - 454ms/epoch - 15ms/step
Epoch 56/100
31/31 - 0s - loss: 172.9367 - 441ms/epoch - 14ms/step
Epoch 57/100
31/31 - 0s - loss: 175.1212 - 437ms/epoch - 14ms/step
Epoch 58/100
31/31 - 0s - loss: 170.2301 - 465ms/epoch - 15ms/step
Epoch 59/100
31/31 - 0s - loss: 169.9245 - 477ms/epoch - 15ms/step
Epoch 60

31/31 - 0s - loss: 148.7488 - 438ms/epoch - 14ms/step
Epoch 68/100
31/31 - 0s - loss: 148.1973 - 425ms/epoch - 14ms/step
Epoch 69/100
31/31 - 0s - loss: 148.9962 - 443ms/epoch - 14ms/step
Epoch 70/100
31/31 - 0s - loss: 146.6479 - 434ms/epoch - 14ms/step
Epoch 71/100
31/31 - 0s - loss: 142.5170 - 470ms/epoch - 15ms/step
Epoch 72/100
31/31 - 0s - loss: 143.6599 - 462ms/epoch - 15ms/step
Epoch 73/100
31/31 - 0s - loss: 145.2834 - 448ms/epoch - 14ms/step
Epoch 74/100
31/31 - 0s - loss: 141.9605 - 436ms/epoch - 14ms/step
Epoch 75/100
31/31 - 0s - loss: 144.8126 - 446ms/epoch - 14ms/step
Epoch 76/100
31/31 - 0s - loss: 143.7653 - 429ms/epoch - 14ms/step
Epoch 77/100
31/31 - 0s - loss: 142.8598 - 435ms/epoch - 14ms/step
Epoch 78/100
31/31 - 0s - loss: 141.5533 - 416ms/epoch - 13ms/step
Epoch 79/100
31/31 - 0s - loss: 141.9357 - 452ms/epoch - 15ms/step
Epoch 80/100
31/31 - 0s - loss: 143.3220 - 468ms/epoch - 15ms/step
Epoch 81/100
31/31 - 0s - loss: 137.1752 - 436ms/epoch - 14ms/step
Epoch 82

31/31 - 0s - loss: 146.5066 - 454ms/epoch - 15ms/step
Epoch 90/100
31/31 - 0s - loss: 149.4747 - 465ms/epoch - 15ms/step
Epoch 91/100
31/31 - 0s - loss: 147.0914 - 480ms/epoch - 15ms/step
Epoch 92/100
31/31 - 0s - loss: 142.9698 - 441ms/epoch - 14ms/step
Epoch 93/100
31/31 - 0s - loss: 144.4787 - 462ms/epoch - 15ms/step
Epoch 94/100
31/31 - 0s - loss: 145.0434 - 490ms/epoch - 16ms/step
Epoch 95/100
31/31 - 0s - loss: 146.2376 - 454ms/epoch - 15ms/step
Epoch 96/100
31/31 - 0s - loss: 142.8915 - 455ms/epoch - 15ms/step
Epoch 97/100
31/31 - 0s - loss: 141.4886 - 475ms/epoch - 15ms/step
Epoch 98/100
31/31 - 0s - loss: 143.9541 - 484ms/epoch - 16ms/step
Epoch 99/100
31/31 - 0s - loss: 143.3945 - 498ms/epoch - 16ms/step
Epoch 100/100
31/31 - 1s - loss: 142.3911 - 508ms/epoch - 16ms/step
16/16 - 0s - 147ms/epoch - 9ms/step
Epoch 1/100
31/31 - 2s - loss: 360.4654 - 2s/epoch - 69ms/step
Epoch 2/100
31/31 - 0s - loss: 315.2877 - 492ms/epoch - 16ms/step
Epoch 3/100
31/31 - 0s - loss: 305.2299 - 4

...

Epoch 77/100
31/31 - 1s - loss: 186.3570 - 500ms/epoch - 16ms/step
Epoch 78/100
31/31 - 0s - loss: 183.1396 - 498ms/epoch - 16ms/step
Epoch 79/100
31/31 - 0s - loss: 184.4939 - 491ms/epoch - 16ms/step
Epoch 80/100
31/31 - 0s - loss: 186.2138 - 479ms/epoch - 15ms/step
Epoch 81/100
31/31 - 0s - loss: 183.9241 - 474ms/epoch - 15ms/step
Epoch 82/100
31/31 - 0s - loss: 181.3950 - 488ms/epoch - 16ms/step
Epoch 83/100
31/31 - 0s - loss: 178.9099 - 471ms/epoch - 15ms/step
Epoch 84/100
31/31 - 0s - loss: 179.8367 - 483ms/epoch - 16ms/step
Epoch 85/100
31/31 - 0s - loss: 180.3710 - 483ms/epoch - 16ms/step
Epoch 86/100
31/31 - 0s - loss: 180.9139 - 483ms/epoch - 16ms/step
Epoch 87/100
31/31 - 0s - loss: 176.8613 - 479ms/epoch - 15ms/step
Epoch 88/100
31/31 - 0s - loss: 177.9234 - 461ms/epoch - 15ms/step
Epoch 89/100
31/31 - 0s - loss: 175.6769 - 467ms/epoch - 15ms/step
Epoch 90/100
31/31 - 0s - loss: 178.1354 - 482ms/epoch - 16ms/step
Epoch 91/100
31/31 - 0s - loss: 178.5654 - 468ms/epoch - 15ms/

Epoch 99/100
31/31 - 0s - loss: 183.0648 - 476ms/epoch - 15ms/step
Epoch 100/100
31/31 - 0s - loss: 182.2477 - 480ms/epoch - 15ms/step
16/16 - 0s - 151ms/epoch - 9ms/step
Epoch 1/100
31/31 - 2s - loss: 370.9902 - 2s/epoch - 80ms/step
Epoch 2/100
31/31 - 0s - loss: 322.4823 - 485ms/epoch - 16ms/step
Epoch 3/100
31/31 - 0s - loss: 314.1855 - 467ms/epoch - 15ms/step
Epoch 4/100
31/31 - 0s - loss: 306.3976 - 487ms/epoch - 16ms/step
Epoch 5/100
31/31 - 0s - loss: 301.6198 - 465ms/epoch - 15ms/step
Epoch 6/100
31/31 - 0s - loss: 297.6531 - 487ms/epoch - 16ms/step
Epoch 7/100
31/31 - 0s - loss: 292.5476 - 474ms/epoch - 15ms/step
Epoch 8/100
31/31 - 0s - loss: 287.1765 - 477ms/epoch - 15ms/step
Epoch 9/100
31/31 - 0s - loss: 283.4911 - 460ms/epoch - 15ms/step
Epoch 10/100
31/31 - 0s - loss: 280.8118 - 472ms/epoch - 15ms/step
Epoch 11/100
31/31 - 0s - loss: 278.0956 - 483ms/epoch - 16ms/step
Epoch 12/100
31/31 - 0s - loss: 273.7501 - 480ms/epoch - 15ms/step
Epoch 13/100
31/31 - 0s - loss: 270.8

Epoch 21/100
31/31 - 0s - loss: 242.5106 - 498ms/epoch - 16ms/step
Epoch 22/100
31/31 - 0s - loss: 242.0970 - 487ms/epoch - 16ms/step
Epoch 23/100
31/31 - 1s - loss: 238.4837 - 508ms/epoch - 16ms/step
Epoch 24/100
31/31 - 1s - loss: 238.6139 - 513ms/epoch - 17ms/step
Epoch 25/100
31/31 - 1s - loss: 236.7068 - 514ms/epoch - 17ms/step
Epoch 26/100
31/31 - 1s - loss: 234.4588 - 502ms/epoch - 16ms/step
Epoch 27/100
31/31 - 0s - loss: 234.9651 - 480ms/epoch - 15ms/step
Epoch 28/100
31/31 - 0s - loss: 231.9933 - 492ms/epoch - 16ms/step
Epoch 29/100
31/31 - 0s - loss: 230.2207 - 471ms/epoch - 15ms/step
Epoch 30/100
31/31 - 0s - loss: 229.3789 - 489ms/epoch - 16ms/step
Epoch 31/100
31/31 - 0s - loss: 230.3928 - 475ms/epoch - 15ms/step
Epoch 32/100
31/31 - 0s - loss: 225.1098 - 474ms/epoch - 15ms/step
Epoch 33/100
31/31 - 1s - loss: 224.2046 - 508ms/epoch - 16ms/step
Epoch 34/100
31/31 - 0s - loss: 223.6073 - 484ms/epoch - 16ms/step
Epoch 35/100
31/31 - 0s - loss: 221.1919 - 470ms/epoch - 15ms/

Epoch 43/100
31/31 - 1s - loss: 221.2775 - 500ms/epoch - 16ms/step
Epoch 44/100
31/31 - 0s - loss: 220.0076 - 495ms/epoch - 16ms/step
Epoch 45/100
31/31 - 0s - loss: 217.6195 - 484ms/epoch - 16ms/step
Epoch 46/100
31/31 - 0s - loss: 212.9045 - 486ms/epoch - 16ms/step
Epoch 47/100
31/31 - 0s - loss: 215.4705 - 481ms/epoch - 16ms/step
Epoch 48/100
31/31 - 0s - loss: 211.2764 - 485ms/epoch - 16ms/step
Epoch 49/100
31/31 - 0s - loss: 215.3639 - 487ms/epoch - 16ms/step
Epoch 50/100
31/31 - 0s - loss: 212.6778 - 498ms/epoch - 16ms/step
Epoch 51/100
31/31 - 0s - loss: 210.2717 - 493ms/epoch - 16ms/step
Epoch 52/100
31/31 - 1s - loss: 210.9669 - 507ms/epoch - 16ms/step
Epoch 53/100
31/31 - 0s - loss: 210.1664 - 485ms/epoch - 16ms/step
Epoch 54/100
31/31 - 0s - loss: 210.3328 - 469ms/epoch - 15ms/step
Epoch 55/100
31/31 - 0s - loss: 210.2599 - 468ms/epoch - 15ms/step
Epoch 56/100
31/31 - 0s - loss: 209.2144 - 487ms/epoch - 16ms/step
Epoch 57/100
31/31 - 0s - loss: 206.2792 - 476ms/epoch - 15ms/

Epoch 65/100
47/47 - 1s - loss: 145.3760 - 727ms/epoch - 15ms/step
Epoch 66/100
47/47 - 1s - loss: 148.7468 - 725ms/epoch - 15ms/step
Epoch 67/100
47/47 - 1s - loss: 146.9155 - 701ms/epoch - 15ms/step
Epoch 68/100
47/47 - 1s - loss: 147.9009 - 709ms/epoch - 15ms/step
Epoch 69/100
47/47 - 1s - loss: 145.6433 - 741ms/epoch - 16ms/step
Epoch 70/100
47/47 - 1s - loss: 146.2180 - 723ms/epoch - 15ms/step
Epoch 71/100
47/47 - 1s - loss: 148.3035 - 728ms/epoch - 15ms/step
Epoch 72/100
47/47 - 1s - loss: 144.2274 - 742ms/epoch - 16ms/step
Epoch 73/100
47/47 - 1s - loss: 145.9633 - 732ms/epoch - 16ms/step
Epoch 74/100
47/47 - 1s - loss: 145.8262 - 740ms/epoch - 16ms/step
Epoch 75/100
47/47 - 1s - loss: 144.6806 - 749ms/epoch - 16ms/step
Epoch 76/100
47/47 - 1s - loss: 143.8346 - 745ms/epoch - 16ms/step
Epoch 77/100
47/47 - 1s - loss: 143.3122 - 703ms/epoch - 15ms/step
Epoch 78/100
47/47 - 1s - loss: 142.8711 - 700ms/epoch - 15ms/step
Epoch 79/100
47/47 - 1s - loss: 141.9641 - 711ms/epoch - 15ms/

Best: -135.957443 using {'model__dropout_rate': 0.0, 'model__weight_constraint': 4}

-145.738597 (0.908516) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1}
-141.332026 (2.666657) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2}
-144.745402 (5.836316) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3}
-135.957443 (7.142423) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4}
-146.625839 (5.329514) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5}
-137.629740 (4.561050) with: {'model__dropout_rate': 0.025, 'model__weight_constraint': 1}
-142.488134 (8.063692) with: {'model__dropout_rate': 0.025, 'model__weight_constraint': 2}
-143.728800 (8.658111) with: {'model__dropout_rate': 0.025, 'model__weight_constraint': 3}
-140.439982 (3.191137) with: {'model__dropout_rate': 0.025, 'model__weight_constraint': 4}
-136.802409 (2.581218) with: {'model__dropout_rate': 0.025, 'model__weight_constraint': 5}
-138.867610 (7.929082) with: {'model__dropout_rate': 0.05, 'model__weight_constraint': 1}
-139.084661 (5.199281) with: {'model__dropout_rate': 0.05, 'model__weight_constraint': 2}
-144.089422 (11.147741) with: {'model__dropout_rate': 0.05, 'model__weight_constraint': 3}
-143.525299 (7.646444) with: {'model__dropout_rate': 0.05, 'model__weight_constraint': 4}
-144.449671 (4.924221) with: {'model__dropout_rate': 0.05, 'model__weight_constraint': 5}
-147.380600 (6.169799) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1}
-143.515533 (5.840852) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2}
-148.573939 (16.722812) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3}
-140.357625 (11.461414) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4}
-145.807927 (5.166111) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5}
-221.601659 (98.880783) with: {'model__dropout_rate': 0.2, 'model__weight_constraint': 1}
-177.990265 (8.557162) with: {'model__dropout_rate': 0.2, 'model__weight_constraint': 2}
-164.588409 (9.139501) with: {'model__dropout_rate': 0.2, 'model__weight_constraint': 3}
-160.114278 (2.219518) with: {'model__dropout_rate': 0.2, 'model__weight_constraint': 4}
-155.914591 (6.280011) with: {'model__dropout_rate': 0.2, 'model__weight_constraint': 5}

In [12]:
#define best model architecture best on hyperparameter tuning
def wider_larger_model(dropout_rate, weight_constraint):
    clear_session()
    model = Sequential()
    del model
    model = Sequential()
    
    model.add(Dense(40, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(60, kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(2)))
    model.add(Dropout(0.05))
    model.add(Dense(50, kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(2)))
    model.add(Dropout(0.05))
    model.add(Dense(30, kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(2)))
    model.add(Dropout(0.05))
    model.add(Dense(20, kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(2)))
    model.add(Dropout(0.05))
    model.add(Dense(10, kernel_initializer='normal', activation='relu', kernel_constraint = MaxNorm(2)))
    model.add(Dropout(0.05))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    # Compile model
    model.compile(loss='mean_squared_error')
    return model

In [13]:
#increase epochs to 200 with best model
model = KerasRegressor(model=wider_larger_model, verbose=2, epochs = 200, batch_size = 1000, optimizer = "Adam",
                      optimizer__learning_rate = 0.1, optimizer__momentum = 0.9, loss = 'mean_squared_error')

# define the grid search parameters
weight_constraint = [4]
dropout_rate = [0.0]
# Make a dictionary of the grid search parameters
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint = weight_constraint )


grid = GridSearchCV(estimator=model, param_grid = param_grid, n_jobs=1, cv= cross_validation, scoring = "neg_mean_squared_error")
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/200
62/62 - 3s - loss: 350.2275 - 3s/epoch - 45ms/step
Epoch 2/200
62/62 - 1s - loss: 312.3590 - 719ms/epoch - 12ms/step
Epoch 3/200
62/62 - 1s - loss: 303.4450 - 767ms/epoch - 12ms/step
Epoch 4/200
62/62 - 1s - loss: 294.8934 - 777ms/epoch - 13ms/step
Epoch 5/200
62/62 - 1s - loss: 287.0010 - 846ms/epoch - 14ms/step
Epoch 6/200
62/62 - 1s - loss: 277.6324 - 946ms/epoch - 15ms/step
Epoch 7/200
62/62 - 1s - loss: 267.4548 - 875ms/epoch - 14ms/step
Epoch 8/200
62/62 - 1s - loss: 260.9442 - 956ms/epoch - 15ms/step
Epoch 9/200
62/62 - 1s - loss: 252.3611 - 938ms/epoch - 15ms/step
Epoch 10/200
62/62 - 1s - loss: 246.7989 - 957ms/epoch - 15ms/step
Epoch 11/200
62/62 - 1s - loss: 239.2553 - 974ms/epoch - 16ms/step
Epoch 12/200
62/62 - 1s - loss: 231.6528 - 953ms/epoch - 15ms/step
Epoch 13/200
62/62 - 1s - loss: 225.9503 - 954ms/epoch - 15ms/step
Epoch 14/200
62/62 - 1s - loss: 222.4599 - 846ms/epoch - 14ms/step
Epoch 15/200
62/62 - 1s - loss: 218.0031 - 826ms/epoch - 13ms/step
Epoch 1

Epoch 124/200
62/62 - 1s - loss: 120.5420 - 874ms/epoch - 14ms/step
Epoch 125/200
62/62 - 1s - loss: 120.0714 - 830ms/epoch - 13ms/step
Epoch 126/200
62/62 - 1s - loss: 120.2601 - 806ms/epoch - 13ms/step
Epoch 127/200
62/62 - 1s - loss: 120.9292 - 822ms/epoch - 13ms/step
Epoch 128/200
62/62 - 1s - loss: 119.2067 - 798ms/epoch - 13ms/step
Epoch 129/200
62/62 - 1s - loss: 120.9803 - 794ms/epoch - 13ms/step
Epoch 130/200
62/62 - 1s - loss: 119.2427 - 818ms/epoch - 13ms/step
Epoch 131/200
62/62 - 1s - loss: 120.4157 - 822ms/epoch - 13ms/step
Epoch 132/200
62/62 - 1s - loss: 119.5320 - 795ms/epoch - 13ms/step
Epoch 133/200
62/62 - 1s - loss: 117.8718 - 810ms/epoch - 13ms/step
Epoch 134/200
62/62 - 1s - loss: 121.0202 - 794ms/epoch - 13ms/step
Epoch 135/200
62/62 - 1s - loss: 120.8393 - 815ms/epoch - 13ms/step
Epoch 136/200
62/62 - 1s - loss: 116.5067 - 805ms/epoch - 13ms/step
Epoch 137/200
62/62 - 1s - loss: 118.2237 - 856ms/epoch - 14ms/step
Epoch 138/200
62/62 - 1s - loss: 120.8240 - 920m

Epoch 45/200
62/62 - 1s - loss: 152.7073 - 823ms/epoch - 13ms/step
Epoch 46/200
62/62 - 1s - loss: 152.3390 - 861ms/epoch - 14ms/step
Epoch 47/200
62/62 - 1s - loss: 153.5080 - 959ms/epoch - 15ms/step
Epoch 48/200
62/62 - 1s - loss: 151.2003 - 998ms/epoch - 16ms/step
Epoch 49/200
62/62 - 1s - loss: 150.3998 - 968ms/epoch - 16ms/step
Epoch 50/200
62/62 - 1s - loss: 151.9395 - 959ms/epoch - 15ms/step
Epoch 51/200
62/62 - 1s - loss: 149.3470 - 971ms/epoch - 16ms/step
Epoch 52/200
62/62 - 1s - loss: 147.8489 - 879ms/epoch - 14ms/step
Epoch 53/200
62/62 - 1s - loss: 149.2628 - 835ms/epoch - 13ms/step
Epoch 54/200
62/62 - 1s - loss: 145.8468 - 808ms/epoch - 13ms/step
Epoch 55/200
62/62 - 1s - loss: 147.7363 - 801ms/epoch - 13ms/step
Epoch 56/200
62/62 - 1s - loss: 144.9236 - 827ms/epoch - 13ms/step
Epoch 57/200
62/62 - 1s - loss: 145.2659 - 822ms/epoch - 13ms/step
Epoch 58/200
62/62 - 1s - loss: 142.2358 - 824ms/epoch - 13ms/step
Epoch 59/200
62/62 - 1s - loss: 141.5596 - 836ms/epoch - 13ms/

Epoch 169/200
62/62 - 1s - loss: 109.1956 - 819ms/epoch - 13ms/step
Epoch 170/200
62/62 - 1s - loss: 110.8424 - 839ms/epoch - 14ms/step
Epoch 171/200
62/62 - 1s - loss: 109.9254 - 807ms/epoch - 13ms/step
Epoch 172/200
62/62 - 1s - loss: 107.9449 - 868ms/epoch - 14ms/step
Epoch 173/200
62/62 - 1s - loss: 109.9020 - 832ms/epoch - 13ms/step
Epoch 174/200
62/62 - 1s - loss: 111.0115 - 844ms/epoch - 14ms/step
Epoch 175/200
62/62 - 1s - loss: 110.4173 - 840ms/epoch - 14ms/step
Epoch 176/200
62/62 - 1s - loss: 109.3448 - 842ms/epoch - 14ms/step
Epoch 177/200
62/62 - 1s - loss: 110.4209 - 839ms/epoch - 14ms/step
Epoch 178/200
62/62 - 1s - loss: 111.0566 - 810ms/epoch - 13ms/step
Epoch 179/200
62/62 - 1s - loss: 109.5788 - 832ms/epoch - 13ms/step
Epoch 180/200
62/62 - 1s - loss: 109.5876 - 821ms/epoch - 13ms/step
Epoch 181/200
62/62 - 1s - loss: 109.3635 - 925ms/epoch - 15ms/step
Epoch 182/200
62/62 - 1s - loss: 108.0298 - 936ms/epoch - 15ms/step
Epoch 183/200
62/62 - 1s - loss: 108.8035 - 961m

Epoch 91/200
62/62 - 1s - loss: 136.4149 - 827ms/epoch - 13ms/step
Epoch 92/200
62/62 - 1s - loss: 135.5632 - 807ms/epoch - 13ms/step
Epoch 93/200
62/62 - 1s - loss: 133.7228 - 834ms/epoch - 13ms/step
Epoch 94/200
62/62 - 1s - loss: 136.7158 - 856ms/epoch - 14ms/step
Epoch 95/200
62/62 - 1s - loss: 133.5096 - 793ms/epoch - 13ms/step
Epoch 96/200
62/62 - 1s - loss: 136.3155 - 824ms/epoch - 13ms/step
Epoch 97/200
62/62 - 1s - loss: 134.9357 - 976ms/epoch - 16ms/step
Epoch 98/200
62/62 - 1s - loss: 133.0649 - 951ms/epoch - 15ms/step
Epoch 99/200
62/62 - 1s - loss: 136.4671 - 947ms/epoch - 15ms/step
Epoch 100/200
62/62 - 1s - loss: 132.2891 - 987ms/epoch - 16ms/step
Epoch 101/200
62/62 - 1s - loss: 134.5533 - 968ms/epoch - 16ms/step
Epoch 102/200
62/62 - 1s - loss: 134.0905 - 950ms/epoch - 15ms/step
Epoch 103/200
62/62 - 1s - loss: 132.3593 - 813ms/epoch - 13ms/step
Epoch 104/200
62/62 - 1s - loss: 131.9138 - 821ms/epoch - 13ms/step
Epoch 105/200
62/62 - 1s - loss: 132.4657 - 821ms/epoch -

93/93 - 1s - loss: 211.7482 - 1s/epoch - 14ms/step
Epoch 13/200
93/93 - 1s - loss: 207.3368 - 1s/epoch - 14ms/step
Epoch 14/200
93/93 - 1s - loss: 203.6177 - 1s/epoch - 13ms/step
Epoch 15/200
93/93 - 1s - loss: 198.2140 - 1s/epoch - 13ms/step
Epoch 16/200
93/93 - 1s - loss: 191.6900 - 1s/epoch - 13ms/step
Epoch 17/200
93/93 - 1s - loss: 188.9237 - 1s/epoch - 16ms/step
Epoch 18/200
93/93 - 1s - loss: 183.0741 - 1s/epoch - 15ms/step
Epoch 19/200
93/93 - 1s - loss: 179.2475 - 1s/epoch - 15ms/step
Epoch 20/200
93/93 - 1s - loss: 177.1015 - 1s/epoch - 15ms/step
Epoch 21/200
93/93 - 1s - loss: 172.4305 - 1s/epoch - 13ms/step
Epoch 22/200
93/93 - 1s - loss: 172.4290 - 1s/epoch - 13ms/step
Epoch 23/200
93/93 - 1s - loss: 170.9032 - 1s/epoch - 13ms/step
Epoch 24/200
93/93 - 1s - loss: 165.9238 - 1s/epoch - 13ms/step
Epoch 25/200
93/93 - 1s - loss: 163.8240 - 1s/epoch - 13ms/step
Epoch 26/200
93/93 - 1s - loss: 163.8321 - 1s/epoch - 13ms/step
Epoch 27/200
93/93 - 1s - loss: 164.4683 - 1s/epoch -

Epoch 140/200
93/93 - 1s - loss: 120.3801 - 1s/epoch - 15ms/step
Epoch 141/200
93/93 - 1s - loss: 120.6225 - 1s/epoch - 15ms/step
Epoch 142/200
93/93 - 1s - loss: 118.9424 - 1s/epoch - 14ms/step
Epoch 143/200
93/93 - 1s - loss: 118.4629 - 1s/epoch - 13ms/step
Epoch 144/200
93/93 - 1s - loss: 119.8030 - 1s/epoch - 13ms/step
Epoch 145/200
93/93 - 1s - loss: 118.8640 - 1s/epoch - 13ms/step
Epoch 146/200
93/93 - 1s - loss: 119.8294 - 1s/epoch - 12ms/step
Epoch 147/200
93/93 - 1s - loss: 119.0325 - 1s/epoch - 13ms/step
Epoch 148/200
93/93 - 1s - loss: 119.3868 - 1s/epoch - 13ms/step
Epoch 149/200
93/93 - 1s - loss: 118.7344 - 1s/epoch - 13ms/step
Epoch 150/200
93/93 - 1s - loss: 118.2268 - 1s/epoch - 13ms/step
Epoch 151/200
93/93 - 1s - loss: 119.7943 - 1s/epoch - 14ms/step
Epoch 152/200
93/93 - 1s - loss: 118.7106 - 1s/epoch - 15ms/step
Epoch 153/200
93/93 - 1s - loss: 119.1345 - 1s/epoch - 15ms/step
Epoch 154/200
93/93 - 1s - loss: 118.9120 - 1s/epoch - 15ms/step
Epoch 155/200
93/93 - 1s 

Best: -122.722730 using {'model__dropout_rate': 0.0, 'model__weight_constraint': 4}
    
-122.722730 (3.486371) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4}

### Model Evaluation

As part of evaluating the created Deep Learning model, we compare its performance to a simple model. 

In [14]:
y_pred_train = grid.predict(X_train)
y_pred_test = grid.predict(X_test)

MSE_train = mean_squared_error(y_train, y_pred_train)
MAE_train = mean_absolute_error(y_train , y_pred_train)
print('Neural Net train MAE = ', MAE_train)
print('Neural Net train MSE = ', MSE_train)

MSE_test = mean_squared_error(y_test, y_pred_test)
MAE_test = mean_absolute_error(y_test , y_pred_test)
print('Neural Net test MAE = ', MAE_test)
print('Neural Net test MSE = ', MSE_test)

93/93 - 0s - 319ms/epoch - 3ms/step
9/9 - 0s - 38ms/epoch - 4ms/step
Neural Net train MAE =  2.910725
Neural Net train MSE =  106.96003
Neural Net test MAE =  2.9867678
Neural Net test MSE =  105.273796


Neural Net train MAE =  2.910725
Neural Net train MSE =  106.96003
Neural Net test MAE =  2.9867678
Neural Net test MSE =  105.273796

We train a Random Forest Regressor as an alternative simple model in order to compare the performance of the final Deep Learning models to this. 

In [26]:
#check alternative model Random Forest Regressor
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)

# Get the mean absolute error on the validation data
y_pred_test = model_rf.predict(X_test)
y_pred_train = model_rf.predict(X_train)

MSE = mean_squared_error(y_train , y_pred_train)
MAE = mean_absolute_error(y_train, y_pred_train)
print('Random forest train MAE = ', MAE)
print('Random forest train MSE = ', MSE)

MSE = mean_squared_error(y_test , y_pred_test)
MAE = mean_absolute_error(y_test, y_pred_test)
print('Random forest test MAE = ', MAE)
print('Random forest test MSE = ', MSE)

Random forest train MAE =  2.146696290844015
Random forest train MSE =  80.72706519833044
Random forest test MAE =  2.56885010203658
Random forest test MSE =  109.4801193196559


In [ ]:
#checking for the curse of dimensionality

import math

number_of_records = df_complete3.shape[0]
number_of_dimensions = df_complete3.shape[1]

print(math.log(number_of_records)/math.log(number_of_dimensions))

In [ ]:
print(math.log(number_of_dimensions)/math.log(number_of_records))